In [1]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

# 2.19 Time series / date functionality

In [5]:
dti = pd.to_datetime(
    ['1.1.2018', np.datetime64('2018-01-01'), datetime.datetime(2018,1,1)]
)
dti[1]

Timestamp('2018-01-01 00:00:00')

In [18]:
dti = pd.date_range('2018-01-01', periods=3, freq='4H')
dti

DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 04:00:00',
               '2018-01-01 08:00:00'],
              dtype='datetime64[ns]', freq='4H')

In [19]:
dti = dti.tz_localize('Europe/Berlin')


DatetimeIndex(['2018-01-01 00:00:00+01:00', '2018-01-01 04:00:00+01:00',
               '2018-01-01 08:00:00+01:00'],
              dtype='datetime64[ns, Europe/Berlin]', freq=None)

In [20]:
dti.tz_convert('US/Central')

DatetimeIndex(['2017-12-31 17:00:00-06:00', '2017-12-31 21:00:00-06:00',
               '2018-01-01 01:00:00-06:00'],
              dtype='datetime64[ns, US/Central]', freq=None)

In [31]:
idx = pd.date_range("2018-01-01", periods=5, freq="H")
ts = pd.Series(range(len(idx)), index=idx)
print(ts)
ts.resample("2H").count()
print(ts.resample("2H").mean())
ts.resample("2H").median()

2018-01-01 00:00:00    0
2018-01-01 01:00:00    1
2018-01-01 02:00:00    2
2018-01-01 03:00:00    3
2018-01-01 04:00:00    4
Freq: H, dtype: int64
2018-01-01 00:00:00    0.5
2018-01-01 02:00:00    2.5
2018-01-01 04:00:00    4.0
Freq: 2H, dtype: float64


2018-01-01 00:00:00    0.5
2018-01-01 02:00:00    2.5
2018-01-01 04:00:00    4.0
Freq: 2H, dtype: float64

In [43]:
friday = pd.Timestamp("2018-01-05")
friday.day_name()
saturday = friday + pd.Timedelta("1 day")
saturday.day_name() # 'Saturday'

# Add 1 business day (Friday --> Monday)
monday = friday + pd.offsets.BDay()
monday.day_name()

'Monday'

In [44]:
pd.Series(range(3), index=pd.date_range('2020', freq='D', periods=3))


2020-01-01    0
2020-01-02    1
2020-01-03    2
Freq: D, dtype: int64

In [45]:
df = pd.DataFrame({"type A":[15], "type B": [20], "date": ["2012-03-01"], "station": ["s1"]})

In [46]:
df

,type A,type B,date,station
0,15,20,2012-03-01,s1


In [9]:
friday = pd.Timestamp("2018-01-05")
friday.day_name()
friday + pd.Timedelta('1 Day')
(friday + pd.offsets.Day()).day_name()

'Saturday'

In [16]:
s = pd.Series(pd.date_range('2020', periods=3))
df = s.to_frame()

In [52]:
df = pd.Series(pd.date_range('1/13/2021', periods=6)).to_frame()
df.columns=["dates"]
df['day'] = df.dates.dt.day_name()
df['weekday'] = df.dates.dt.weekday
df

,dates,day,weekday
0,2021-01-13,Wednesday,2
1,2021-01-14,Thursday,3
2,2021-01-15,Friday,4
3,2021-01-16,Saturday,5
4,2021-01-17,Sunday,6
5,2021-01-18,Monday,0


In [61]:
from pandas.tseries.offsets import Week

print(df['dates'] - Week(1, weekday=2))
df['dates'] - pd.offsets.Week(1, weekday=2)

df['pday'] = np.where(df.dates.eq('2021-01-15'),
         df.dates - pd.offsets.Week(2, weekday=2),
        df.dates - pd.offsets.Week(weekday=2))
df

0   2021-01-06
1   2021-01-13
2   2021-01-13
3   2021-01-13
4   2021-01-13
5   2021-01-13
Name: dates, dtype: datetime64[ns]


,dates,day,weekday,pday
0,2021-01-13,Wednesday,2,2021-01-06
1,2021-01-14,Thursday,3,2021-01-13
2,2021-01-15,Friday,4,2021-01-06
3,2021-01-16,Saturday,5,2021-01-13
4,2021-01-17,Sunday,6,2021-01-13
5,2021-01-18,Monday,0,2021-01-13


In [65]:
df=pd.DataFrame({'Date':['2020-01-01', '2020-01-23', '2020-02-22', '2020-03-04'],  'ID':[1001,1002,1002,1003]})
df["Date"] = pd.to_datetime(df["Date"])
df['Wed'] = np.where(df.ID == 1002,
         df.Date - pd.offsets.Week(3, weekday=2),
         df.Date - pd.offsets.Week(weekday=2))
df

,Date,ID,Wed
0,2020-01-01,1001,2019-12-25
1,2020-01-23,1002,2020-01-08
2,2020-02-22,1002,2020-02-05
3,2020-03-04,1003,2020-02-26


In [79]:
data = """Date, Close,Upper,SMA,Lower,Buy,Sell
2020-05-21,231.389999,219.042175,207.178002,195.313829,NaN,231.389999 
2020-05-22,234.910004,222.051354,209.420002,196.788650,NaN,234.910004
2020-05-26,232.199997,224.164115,211.655002,199.145889,NaN,NaN
2020-05-27,229.139999,225.151643,213.966502,202.781360,NaN,NaN
2020-05-28,225.460007,226.004370,215.530002,205.055633,NaN,NaN
2020-05-29,225.089996,226.911921,216.549001,206.186082,NaN,NaN
2020-06-01,231.910004,228.365279,218.031001,207.696723,NaN,NaN"""
df = pd.read_csv(StringIO(data))
df = df.set_index('Date')
df

,Close,Upper,SMA,Lower,Buy,Sell
Date,,,,,,
2020-05-21,231.389999,219.042175,207.178002,195.313829,NaN,231.389999
2020-05-22,234.910004,222.051354,209.420002,196.788650,NaN,234.910004
2020-05-26,232.199997,224.164115,211.655002,199.145889,NaN,NaN
2020-05-27,229.139999,225.151643,213.966502,202.781360,NaN,NaN
2020-05-28,225.460007,226.004370,215.530002,205.055633,NaN,NaN
2020-05-29,225.089996,226.911921,216.549001,206.186082,NaN,NaN
2020-06-01,231.910004,228.365279,218.031001,207.696723,NaN,NaN


In [85]:
df.query('Sell.notna() | Buy.notna()')

,Close,Upper,SMA,Lower,Buy,Sell
Date,,,,,,
2020-05-21,231.389999,219.042175,207.178002,195.313829,NaN,231.389999
2020-05-22,234.910004,222.051354,209.420002,196.788650,NaN,234.910004


In [130]:
data="""
ID,name,date,confirmedInfections
DE2,BAYERN,2020-02-24,19
DE2,BAYERN,2020-02-25,19
DE2,BAYERN,2020-02-26,21
DE1,BADEN-WÃRTTEMBERG,2020-02-24,1
DE1,BADEN-WÃRTTEMBERG,2020-02-25,3
DE1,BADEN-WÃRTTEMBERG,2020-02-26,7
"""
df = pd.read_csv(StringIO(data))
df

,ID,name,date,confirmedInfections
0,DE2,BAYERN,2020-02-24,19
1,DE2,BAYERN,2020-02-25,19
2,DE2,BAYERN,2020-02-26,21
3,DE1,BADEN-WÃRTTEMBERG,2020-02-24,1
4,DE1,BADEN-WÃRTTEMBERG,2020-02-25,3
5,DE1,BADEN-WÃRTTEMBERG,2020-02-26,7


In [131]:
df["confirmedInfections"] = df.groupby('ID')['confirmedInfections'].diff().fillna(df.confirmedInfections).astype(int)


In [132]:
for id in df.ID.unique():
    print('DataFrame', id)
    print(df.groupby('ID').get_group(id))
    print()

DataFrame DE2
    ID    name        date  confirmedInfections
0  DE2  BAYERN  2020-02-24                   19
1  DE2  BAYERN  2020-02-25                    0
2  DE2  BAYERN  2020-02-26                    2

DataFrame DE1
    ID               name        date  confirmedInfections
3  DE1  BADEN-WÃRTTEMBERG  2020-02-24                    1
4  DE1  BADEN-WÃRTTEMBERG  2020-02-25                    2
5  DE1  BADEN-WÃRTTEMBERG  2020-02-26                    4



In [135]:
df1 = pd.DataFrame({'ID': ['AAA','AAA', 'AAA','BBB','CCC'], 'Buydate': ['2017-04-13', '2019-12-31', '2019-03-05', '2018-11-04', '2019-12-31' ], 'Quantity': [100.00, 2000.00, 385.95, 214514.00, 63205.00]}) 
df1 = df1.sort_values('Buydate')
#df1 = df1.set_index(['ID', 'Buydate']).reset_index()
df1['Buydate'] = pd.to_datetime(df1.Buydate)
df1

,ID,Buydate,Quantity
0,AAA,2017-04-13,100.00
3,BBB,2018-11-04,214514.00
2,AAA,2019-03-05,385.95
1,AAA,2019-12-31,2000.00
4,CCC,2019-12-31,63205.00


In [64]:
df2=pd.DataFrame({'ID': ['AAA','AAA','BBB'],
               'Selldate': ['2020-01-25', '2020-10-25', '2020-12-19'],
               'Quantity': [500.00,  1985.95, 214714.00]})
#df2 = df2.set_index(['ID', 'Selldate']).reset_index()
df2 = df2.sort_values('Selldate')
df2['Selldate'] = pd.to_datetime(df2.Selldate)
df2

,ID,Selldate,Quantity
0,AAA,2020-01-25,500.00
1,AAA,2020-10-25,1985.95
2,BBB,2020-12-19,214714.00


In [87]:
df = pd.merge_asof(df1, df2, by='ID',
            left_on='Buydate', right_on='Selldate', direction='forward').rename(
    columns={'Quantity_x':'BuyQuantity', 'Quantity_y':'SellQuantity'}).sort_values(
    'ID').reset_index(drop=True)
df

,ID,Buydate,BuyQuantity,Selldate,SellQuantity
0,AAA,2017-04-13,100.00,2020-01-25,500.0
1,AAA,2019-03-05,385.95,2020-01-25,500.0
2,AAA,2019-12-31,2000.00,2020-01-25,500.0
3,BBB,2018-11-04,214514.00,2020-12-19,214714.0
4,CCC,2019-12-31,63205.00,NaT,NaN


In [90]:
df['sq'] = np.where(df.SellQuantity > df.BuyQuantity,
                   df.apply(lambda row: row.name+1, axis=1),
                   df.BuyQuantity)
df

df.append(pd.DataFrame([df.iloc[df.apply(lambda row: row.name, axis=1)]],index=[df.apply(lambda row: row.name+1, axis=1)],columns=df.columns))

,ID,Buydate,BuyQuantity,Selldate,SellQuantity,sq
0,AAA,2017-04-13,100.00,2020-01-25,500.0,1.0
1,AAA,2019-03-05,385.95,2020-01-25,500.0,2.0
2,AAA,2019-12-31,2000.00,2020-01-25,500.0,2000.0
3,BBB,2018-11-04,214514.00,2020-12-19,214714.0,4.0
4,CCC,2019-12-31,63205.00,NaT,NaN,63205.0


In [102]:
data = """Model,Checkpoint,ROUGE_L
4,1005100,0.204
4,1010200,0.202
4,1015300,0.205
4,1020400,0.203
4,1025500,0.204
16000,1030600,0.396
16000,1035700,0.396
16000,1040800,0.408"""

df = pd.read_csv(StringIO(data))
df

,Model,Checkpoint,ROUGE_L
0,4,1005100,0.204
1,4,1010200,0.202
2,4,1015300,0.205
3,4,1020400,0.203
4,4,1025500,0.204
5,16000,1030600,0.396
6,16000,1035700,0.396
7,16000,1040800,0.408


In [103]:
df.groupby(['Model','Checkpoint'])['ROUGE_L'].max()

Model  Checkpoint
4      1005100       0.204
       1010200       0.202
       1015300       0.205
       1020400       0.203
       1025500       0.204
16000  1030600       0.396
       1035700       0.396
       1040800       0.408
Name: ROUGE_L, dtype: float64

In [129]:
%timeit df.set_index('ROUGE_L').sort_index().reset_index().groupby(['Model'], as_index=False).last()

3.36 ms ± 64.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [128]:
%timeit df.loc[df.groupby("Model")["ROUGE_L"].idxmax()]

1.44 ms ± 35.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [130]:
df.groupby('Model')["ROUGE_L"].max()

Model
4        0.205
16000    0.408
Name: ROUGE_L, dtype: float64

In [133]:
df.groupby("Model")["ROUGE_L"].idxmax()

Model
4        2
16000    7
Name: ROUGE_L, dtype: int64

In [134]:
df.set_index('ROUGE_L').sort_index().reset_index().groupby(['Model'], as_index=False).first()

,Model,ROUGE_L,Checkpoint
0,4,0.202,1010200
1,16000,0.396,1030600


In [157]:
df1 = pd.DataFrame({'ID': ['AAA','AAA', 'AAA','BBB','CCC'], 'Buydate': ['2019-04-13', '2019-5-31', '2019-03-05', '2019-3-04', '2019-7-31' ], 'Quantity': [100.00, 2000.00, 385.95, 214514.00, 63205.00]}) 
df1 = df1.sort_values('Buydate')
#df1 = df1.set_index(['ID', 'Buydate']).reset_index()
df1['Buydate'] = pd.to_datetime(df1.Buydate)
df1

,ID,Buydate,Quantity
2,AAA,2019-03-05,385.95
0,AAA,2019-04-13,100.00
3,BBB,2019-03-04,214514.00
1,AAA,2019-05-31,2000.00
4,CCC,2019-07-31,63205.00


In [158]:
df1.drop(columns='ID', inplace=True)

In [159]:
df1.set_index('Buydate', inplace=True)

In [160]:
df1.reset_index()

,Buydate,Quantity
0,2019-03-05,385.95
1,2019-04-13,100.00
2,2019-03-04,214514.00
3,2019-05-31,2000.00
4,2019-07-31,63205.00


In [167]:
g = df1.groupby(pd.Grouper(freq='M')).sum()
g.reset_index(inplace=True)
g

,Buydate,Quantity
0,2019-03-31,214899.95
1,2019-04-30,100.00
2,2019-05-31,2000.00
3,2019-06-30,0.00
4,2019-07-31,63205.00


In [184]:
df = pd.DataFrame({"col_1": ["apple", "banana", "apple", "banana", "banana"],
                   "col_2": [1, 4, 8, 8, 6],
                   "col_3": [56, 4, 22, 1, 5]})
for i in range(20):
    df = pd.concat([df,df])

df.shape

(5242880, 3)

In [185]:
%timeit df.groupby('col_1').agg({ 'col_2': lambda x: list(x),'col_3': lambda x: list(x),}).reset_index().join(df.groupby('col_1')['col_2'].transform('size').rename('count'))

3.54 s ± 53.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [186]:
%timeit df.groupby('col_1').agg({ 'col_2': lambda x: list(x),'col_3': lambda x: list(x),}).reset_index().join(df.groupby('col_1')['col_2'].transform('count').rename('count'))

3.55 s ± 26.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
pd.Timestamp(pd.NaT)
pd.Timedelta(pd.NaT)
pd.Period(pd.NaT)
pd.DateOffset()

nan